In [2]:
import sys
import torch
from torch import nn
import torchvision.models as models
import time
from tqdm import tqdm
import torch.optim as optim
import copy

# Specify where to find the data preparation class
sys.path.append('../../Data_Preparation')
from Preparation import CustomDataLoader

In [3]:
# InceptionV3 training data (ImageNet) properties
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
DIMENSIONS = 3
BATCH_SIZE = 16

LR = 0.1
MOMENTUM=0.9
WEIGHT_DECAY = 1e-4
LR_STEP_SIZE = 30
LR_GAMMA = 0.1

EPOCHS = 90

In [4]:
print("Creating data loaders")
# Instantiate the CustomDataLoader class for training
train_data_loader = CustomDataLoader(data_path="../../FER2013_Data", batch_size=BATCH_SIZE, dataset_type="train", mean=MEAN, std=STD, dimensions=3).data_loader
test_data_loader = CustomDataLoader(data_path="../../FER2013_Data", batch_size=BATCH_SIZE, dataset_type="test", mean=MEAN, std=STD, dimensions=3).data_loader

# Confirm correct data load
print("Train Data Loader:")
for batch_idx, (inputs, labels) in enumerate(train_data_loader):
    print("Batch Index:", batch_idx)
    print("Inputs Shape:", inputs.shape)
    print("Labels Shape:", labels.shape)
    # Print the first few labels in the batch
    print("Labels:", labels[:5])
    # Break after printing a few batches
    if batch_idx == 2:
        break

Creating data loaders
Train Data Loader:
Batch Index: 0
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([3, 0, 3, 0, 0])
Batch Index: 1
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([4, 6, 3, 0, 4])
Batch Index: 2
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([0, 6, 2, 4, 3])


In [4]:
# load up the InceptionV3 model
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model = models.inception_v3(pretrained=True)
model.eval()

for parameter in model.parameters():
    parameter.requires_grad = False

# Replace the last fully connected layer with a new one that outputs 7 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)  # Output layer with 7 classes
model.aux_logits = False
model.AuxLogits = None

model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=LR,
    momentum=MOMENTUM,
    weight_decay=WEIGHT_DECAY
)

LR_SCHEDULER = torch.optim.lr_scheduler.StepLR(optimizer, step_size=LR_STEP_SIZE, gamma=LR_GAMMA)

#torch.optim.Adam(model.parameters(), lr=0.0001)

C:\users\kh2197vi\downloads\facial_emotion_detec-main\facial_emotion_detec-main\.env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\users\kh2197vi\downloads\facial_emotion_detec-main\facial_emotion_detec-main\.env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
print("[INFO] Start training")
print("---------------------")

for epoch in tqdm(range(EPOCHS)):

    total_batch = len(train_data_loader.dataset)//BATCH_SIZE
    
    # Training phase
    model.train()  # Set the model to train mode

    # Get statistics
    epoch_loss = 0
    len_dataset = 0
    
    for step, (batch_images, batch_labels) in enumerate(train_data_loader):
        X, Y = batch_images.to(device), batch_labels.to(device)

        pred = model(X)
        loss = criterion(pred, Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += pred.shape[0] * loss.item()
        len_dataset += pred.shape[0]
        if (step) % 10 == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                 %(epoch+1, EPOCHS, step+1, total_batch, loss.item()))

    epoch_loss = epoch_loss/ len_dataset
    print('Epoch: ', epoch+1, '| train loss : %0.4f' % epoch_loss)

    LR_SCHEDULER.step()
    
    # Validation phase
    model.eval()  # Set the model to evaluation mode
    with torch.inference_mode():  
        running_loss = 0
        for step, (images, labels) in enumerate(test_data_loader):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            
    running_loss = running_loss / len(test_data_loader)
    print('Epoch: ', epoch, '| test loss : %0.4f' % running_loss )


[INFO] Start training
---------------------


  0%|          | 0/90 [00:00<?, ?it/s]

Epoch [1/90], lter [1/1752] Loss: 1.9066
Epoch [1/90], lter [11/1752] Loss: 9.6600
Epoch [1/90], lter [21/1752] Loss: 15.6475
Epoch [1/90], lter [31/1752] Loss: 22.2051
Epoch [1/90], lter [41/1752] Loss: 17.0985
Epoch [1/90], lter [51/1752] Loss: 15.4631
Epoch [1/90], lter [61/1752] Loss: 14.0895
Epoch [1/90], lter [71/1752] Loss: 15.2768
Epoch [1/90], lter [81/1752] Loss: 5.2184
Epoch [1/90], lter [91/1752] Loss: 12.1279
Epoch [1/90], lter [101/1752] Loss: 19.4472
Epoch [1/90], lter [111/1752] Loss: 11.7660
Epoch [1/90], lter [121/1752] Loss: 12.4745
Epoch [1/90], lter [131/1752] Loss: 19.8807
Epoch [1/90], lter [141/1752] Loss: 19.9148
Epoch [1/90], lter [151/1752] Loss: 10.7936
Epoch [1/90], lter [161/1752] Loss: 15.7218
Epoch [1/90], lter [171/1752] Loss: 15.2498
Epoch [1/90], lter [181/1752] Loss: 28.2794
Epoch [1/90], lter [191/1752] Loss: 19.8906
Epoch [1/90], lter [201/1752] Loss: 20.7210
Epoch [1/90], lter [211/1752] Loss: 8.7459
Epoch [1/90], lter [221/1752] Loss: 16.9010
Epo

  1%|          | 1/90 [62:48:38<5590:09:47, 226118.96s/it]

Epoch:  0 | test loss : 20.6425
Epoch [2/90], lter [1/1752] Loss: 25.6945
Epoch [2/90], lter [11/1752] Loss: 26.5892
Epoch [2/90], lter [21/1752] Loss: 27.3370
Epoch [2/90], lter [31/1752] Loss: 17.6685
Epoch [2/90], lter [41/1752] Loss: 27.7771
Epoch [2/90], lter [51/1752] Loss: 13.9643
Epoch [2/90], lter [61/1752] Loss: 23.7278
Epoch [2/90], lter [71/1752] Loss: 28.5910
Epoch [2/90], lter [81/1752] Loss: 20.5958
Epoch [2/90], lter [91/1752] Loss: 19.4352
Epoch [2/90], lter [101/1752] Loss: 24.2826
Epoch [2/90], lter [111/1752] Loss: 26.2649
Epoch [2/90], lter [121/1752] Loss: 26.7614
Epoch [2/90], lter [131/1752] Loss: 22.5411
Epoch [2/90], lter [141/1752] Loss: 12.3127
Epoch [2/90], lter [151/1752] Loss: 22.5944
Epoch [2/90], lter [161/1752] Loss: 24.9117
Epoch [2/90], lter [171/1752] Loss: 14.9085
Epoch [2/90], lter [181/1752] Loss: 26.0958
Epoch [2/90], lter [191/1752] Loss: 21.8663
Epoch [2/90], lter [201/1752] Loss: 28.5280
Epoch [2/90], lter [211/1752] Loss: 20.4405
Epoch [2/90

...

Epoch:  87 | test loss : 1.5497
Epoch [89/90], lter [1/1752] Loss: 1.6617
Epoch [89/90], lter [11/1752] Loss: 1.4659
Epoch [89/90], lter [21/1752] Loss: 1.4132
Epoch [89/90], lter [31/1752] Loss: 1.9695
Epoch [89/90], lter [41/1752] Loss: 1.7819
Epoch [89/90], lter [51/1752] Loss: 1.7256
Epoch [89/90], lter [61/1752] Loss: 1.2862
Epoch [89/90], lter [71/1752] Loss: 2.2512
Epoch [89/90], lter [81/1752] Loss: 1.4509
Epoch [89/90], lter [91/1752] Loss: 1.6939
Epoch [89/90], lter [101/1752] Loss: 1.6550
Epoch [89/90], lter [111/1752] Loss: 1.5661
Epoch [89/90], lter [121/1752] Loss: 1.3326
Epoch [89/90], lter [131/1752] Loss: 1.2978
Epoch [89/90], lter [141/1752] Loss: 1.5035
Epoch [89/90], lter [151/1752] Loss: 1.5776
Epoch [89/90], lter [161/1752] Loss: 1.7118
Epoch [89/90], lter [171/1752] Loss: 1.4298
Epoch [89/90], lter [181/1752] Loss: 1.5735
Epoch [89/90], lter [191/1752] Loss: 1.3604
Epoch [89/90], lter [201/1752] Loss: 1.6622
Epoch [89/90], lter [211/1752] Loss: 2.1149
Epoch [89/9

 99%|█████████▉| 89/90 [89:57:26<23:03, 1383.24s/it]

Epoch:  88 | test loss : 1.5536
Epoch [90/90], lter [1/1752] Loss: 1.7849
Epoch [90/90], lter [11/1752] Loss: 1.3901
Epoch [90/90], lter [21/1752] Loss: 1.5012
Epoch [90/90], lter [31/1752] Loss: 1.5726
Epoch [90/90], lter [41/1752] Loss: 1.8749
Epoch [90/90], lter [51/1752] Loss: 1.4391
Epoch [90/90], lter [61/1752] Loss: 1.3837
Epoch [90/90], lter [71/1752] Loss: 1.3433
Epoch [90/90], lter [81/1752] Loss: 1.8068
Epoch [90/90], lter [91/1752] Loss: 1.7876
Epoch [90/90], lter [101/1752] Loss: 1.8290
Epoch [90/90], lter [111/1752] Loss: 1.7019
Epoch [90/90], lter [121/1752] Loss: 1.9735
Epoch [90/90], lter [131/1752] Loss: 1.7826
Epoch [90/90], lter [141/1752] Loss: 1.3876
Epoch [90/90], lter [151/1752] Loss: 2.0638
Epoch [90/90], lter [161/1752] Loss: 1.8288
Epoch [90/90], lter [171/1752] Loss: 1.3867
Epoch [90/90], lter [181/1752] Loss: 1.5316
Epoch [90/90], lter [191/1752] Loss: 1.6545
Epoch [90/90], lter [201/1752] Loss: 1.6986
Epoch [90/90], lter [211/1752] Loss: 1.8110
Epoch [90/9

100%|██████████| 90/90 [90:20:34<00:00, 3613.72s/it]

Epoch:  89 | test loss : 1.5466


In [6]:
# Saving the model
torch.save(model.state_dict(), 'trained_inception_v3.pt')

In [10]:
model = torch.load('trained_inception_v3.pt', map_location=torch.device('cpu'), weights_only=True)

C:\users\kh2197vi\downloads\facial_emotion_detec-main\facial_emotion_detec-main\.env\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [12]:
# Specify the path to the saved model
PATH = 'trained_inception_v3.pt'

# Load the model
model = models.inception_v3(pretrained=True)

for parameter in model.parameters():
    parameter.requires_grad = False

# Replace the last fully connected layer with a new one that outputs 7 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)  # Output layer with 7 classes
model.aux_logits = False
model.AuxLogits = None

# Load the saved weights
model_state_dict = torch.load(PATH)

# Load the state_dict into the model
model.load_state_dict(model_state_dict)

# Set the model to evaluation mode
model.eval()

C:\users\kh2197vi\downloads\facial_emotion_detec-main\facial_emotion_detec-main\.env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\users\kh2197vi\downloads\facial_emotion_detec-main\facial_emotion_detec-main\.env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [13]:
# Initialize the correct_predictions and total_predictions variables
correct_predictions, total_predictions = 0, 0

# Loop over the batches of the test set
for batch_data in test_data_loader:
    # Forward pass
    inputs, target = batch_data
    output = model(inputs)

    # Get the predicted class
    _, predicted = torch.max(output.data, 1)

    # Increase the total predictions counter
    total_predictions += target.size(0)

    # Check if the predicted class is equal to the true class
    correct_predictions += (predicted == target).sum().item()

# Calculate the accuracy
accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 38.87%
